# Tropicalization of a p-adic surface

Inspired by https://github.com/emresertoz/pAdicCubicSurface

Author: Michael Joswig

In [1]:
using Oscar

$\require{action}$

┌ Info: Precompiling Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13]
└ @ Base loading.jl:1260
[ Info: Generating module common
[ Info: Generating module ideal
[ Info: Generating module graph
[ Info: Generating module fulton
[ Info: Generating module fan
[ Info: Generating module group
[ Info: Generating module polytope
[ Info: Generating module topaz
[ Info: Generating module tropical
[ Info: Generating module matroid


 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) GAP, Hecke, Nemo, Polymake and Singular
Version 0.2.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2020 by The Oscar Development Team


In [2]:
import Pkg
Pkg.status()

Status `~/.julia/environments/v1.4/Project.toml`
  [b4f34e82] Distances v0.8.2
  [7073ff75] IJulia v1.21.1
  [f1435218] Oscar v0.2.0
  [91a5bcdd] Plots v1.0.10
  [d720cf60] Polymake v0.3.3


We start out with basic arithmetic in a p-adic field; this comes from FLINT.

In [3]:
p=5
Qp = PadicField(p, 30)

a = 2*p + 4*p^2 + 1*p^3 + O(Qp, p^4);

@show a
@show valuation(a);

a = 2*5^1 + 4*5^2 + 1*5^3 + O(5^4)
valuation(a) = 1


The polynomials come from AbstractAlgebra.jl.

In [4]:
R, (w,x,y,z) = PolynomialRing(Qp, ["w","x","y","z"])

f = 3125*w^3 + 25*w^2*x + 25*w^2*y + 5*w^2*z + 25*w*x^2 + w*x*y + w*x*z + 25*w*y^2 + w*y*z + 5*w*z^2 + 3125*x^3 + 5*x^2*y + 25*x^2*z + 5*x*y^2 + x*y*z + 25*x*z^2 + 3125*y^3 + 25*y^2*z + 25*y*z^2 + 3125*z^3;

Now we can translate this into a function which employs polymake.  Note that the classical p-adic valuation naturally forces $\min$ as the tropical addition.

In [5]:
function tropical_hypersurface(f)
    C = map(c->Int(valuation(c)), coeffs(f))
    E = transpose(hcat(collect(Nemo.exponent_vectors(f))...))
    return Polymake.@pm tropical.Hypersurface{Min}(COEFFICIENTS=C, MONOMIALS=E)
end

tropical_hypersurface (generic function with 1 method)

Indeed, the resulting tropical hypersurface is cubic.

In [6]:
H = tropical_hypersurface(f)

@show H.DEGREE

H.DEGREE = 3


3

In [7]:
D = H.DUAL_SUBDIVISION

type: SubdivisionOfPoints<Rational>

POINTS
  1  3  0  0  0
  1  2  1  0  0
  1  2  0  1  0
  1  2  0  0  1
  1  1  2  0  0
  1  1  1  1  0
  1  1  1  0  1
  1  1  0  2  0
  1  1  0  1  1
  1  1  0  0  2
  1  0  3  0  0
  1  0  2  1  0
  1  0  2  0  1
  1  0  1  2  0
  1  0  1  1  1
  1  0  1  0  2
  1  0  0  3  0
  1  0  0  2  1
  1  0  0  1  2
  1  0  0  0  3

WEIGHTS
	5 2 2 1 2 0 0 2 0 1 5 1 2 1 0 2 5 2 2 5

polymake's bundled extension <tt>a-tint</tt> provides a function for finding lines on a tropical cubic surface.  In contrast to the classical setting there may be infinite families.

In [8]:
L = Polymake.tropical.lines_in_cubic(H.POLYNOMIAL);

@show L.N_ISOLATED
@show L.N_FAMILIES

L.N_ISOLATED = 16
L.N_FAMILIES = 43


43

In [9]:
L, t = PuiseuxSeriesRing(QQ, 30, "t")

(Puiseux series field in t over Rational Field, t+O(t^31))

In [10]:
s = -t^3+t^16
valuation(s)

3//1

In [11]:
function leading_coefficient(series)
    return coeff(series, Int(valuation(series)))
end

function sign_leading_coefficient(series)
    lc = leading_coefficient(series)
    if lc < 0
        return 1
    else
        return 0
    end
end

function evaluate_puiseux_series(series, t)
    s = series.scale
    y = root(t, s)
    v = valuation(series)
    p = precision(series)
    z = zero(y)
    for i = Int(p*s):-1:Int(s*v)
        c = coeff(series, i//s)
        if !iszero(c)
            z += c
        end
        z *= y
    end
    z *= y^Int(s*v-1)
    return z
end

evaluate_puiseux_series (generic function with 1 method)

In [12]:
@show leading_coefficient(s)
@show sign_leading_coefficient(s);

leading_coefficient(s) = -1
sign_leading_coefficient(s) = 1


In [13]:
RR = Nemo.ArbField(30)

@show real_s = evaluate_puiseux_series(s, RR(0.00001))
convert(Float64, Nemo.midpoint(real_s))

real_s = evaluate_puiseux_series(s, RR(1.0e-5)) = [-1.000000e-15 +/- 1.57e-23]


-9.999999937013264e-16

In [14]:
R, (x,y,z) = PolynomialRing(L, ["x","y","z"])

harnack = x^3 - t*x^2*y + t*x^2*z - t^4*x*y^2 - t^3*x*y*z - t^4*x*z^2 - t^9*y^3 + t^7*y^2*z - t^7*y*z^2 - t^9*z^3

x^3-t+O(t^31)*x^2*y+t+O(t^31)*x^2*z-t^4+O(t^34)*x*y^2-t^3+O(t^33)*x*y*z-t^4+O(t^34)*x*z^2-t^9+O(t^39)*y^3+t^7+O(t^37)*y^2*z-t^7+O(t^37)*y*z^2-t^9+O(t^39)*z^3

In [15]:
H = tropical_hypersurface(harnack)

@show H.DEGREE

H.DEGREE = 3


3

In [16]:
R_RR, (x,y,z) = PolynomialRing(RR, ["x","y","z"])

(Multivariate Polynomial Ring in x, y, z over Real Field with 30 bits of precision and error bounds, AbstractAlgebra.Generic.MPoly{arb}[x, y, z])

In [17]:
Pkg.add("HomotopyContinuation")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


    Fetching: [=======>

    Fetching: [=============>                           ]  31.0

Fetching: [========================================>]  100.0 %                    ]  47.1 % [==========================>              ]  62.7 %78.1 %9 %.2 %

  Resolving package versions...
  Installed Opus_jll ────── v1.3.1+1
  Installed libvorbis_jll ─ v1.3.6+3
  Installed DoubleFloats ── v1.1.9
######################################################################### 100,0%##O=#  #                                                                       
######################################################################### 100,0%##O#- #                                                                        
######################################################################### 100,0%#=#=#                                                                          
######################################################################### 100,0%#=#=#                                                                          
   Updating `~/.julia/environments/v1.4/Project.toml`
  [f213a82b] + HomotopyContinuation v1.4.1
   Updating `~/.julia/environments/v1.4/Manifest.toml`
  [e66e0078] + CompilerSupportLibraries_jll v0.3.3+0
  [a8cc5b0e] +

In [18]:
import HomotopyContinuation

function hc_poly(f, t, vars)
    M = transpose(hcat(collect(Nemo.exponent_vectors(f))...))
    monomials_as_matrix = [prod(vars.^m) for m in eachrow(M)]
    coeffs = map(c->convert(Float64, Nemo.midpoint(evaluate_puiseux_series(c,t))), coeffs(f))
    sum(map(*, coeffs, monomials_as_matrix))
end;

┌ Info: Precompiling HomotopyContinuation [f213a82b-91d6-5c5d-acf7-10f1c761b327]
└ @ Base loading.jl:1260
┌ Warning: Use of `Poly` from v1.0 forward will require `using Polynomials.PolyCompat`
│   caller = Polynomials.PolyCompat.Poly(::Array{Double64,1}) at Poly.jl:26
└ @ Polynomials.PolyCompat ~/.julia/packages/Polynomials/ghqdF/src/polynomials/Poly.jl:26


In [19]:
function real_tropical_hypersurface(f)
    coeffs_f = coeffs(f)
    C = map(c->Int(valuation(c)), coeffs_f)
    E = transpose(hcat(collect(Nemo.exponent_vectors(f))...))
    H = Polymake.@pm tropical.Hypersurface{Min}(COEFFICIENTS=C, MONOMIALS=E)
    S = map(c->Int(sign_leading_coefficient(c)), coeffs_f)
    H.PATCHWORK(SIGNS=S)
    return H
end

real_tropical_hypersurface (generic function with 1 method)

In [20]:
real_tropical_hypersurface(harnack)

polymake:  WARNING: available properties insufficient to compute 'PATCHWORK'


Polymake.PolymakeError: Exception occured at Polymake side:
property PATCHWORK not created as expected at /home/mic/.julia/packages/Polymake/asr90/deps/usr/share/polymake/perllib/Polymake/Core/BigObject.pm line 1559.
